<a href="https://colab.research.google.com/github/EyadNasr/Coursera-Courses-to-Google-Drive/blob/master/Coursera_Courses_to_Google_Drive.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@markdown <br><center><img src='https://upload.wikimedia.org/wikipedia/commons/thumb/d/da/Google_Drive_logo.png/600px-Google_Drive_logo.png' height="50" alt="Gdrive-logo"/></center>
#@markdown <center><h3>Mount GDrive to /content/drive</h3></center><br>
MODE = "MOUNT" #@param ["MOUNT", "UNMOUNT"]
#Mount your Gdrive! 
from google.colab import drive
drive.mount._DEBUG = False
if MODE == "MOUNT":
  drive.mount('/content/drive', force_remount=True)
elif MODE == "UNMOUNT":
  try:
    drive.flush_and_unmount()
  except ValueError:
    pass
  get_ipython().system_raw("rm -rf /root/.config/Google/DriveFS")

In [ ]:

#@markdown <br><center><img src='https://upload.wikimedia.org/wikipedia/en/3/32/Coursera_logo.svg' height="50"/> <font color="black"><h> To </h></font> <img src='https://upload.wikimedia.org/wikipedia/commons/thumb/d/da/Google_Drive_logo.png/600px-Google_Drive_logo.png' height="50" alt="Gdrive-logo"/>

#@markdown <br><center><h3>Download Coursera Courses to Google Drive</h3></center><br>
import os
try : import coursera
except :
  print('Installing coursera-dl.....')
  import subprocess
  subprocess.call(['pip', 'install', 'coursera-dl'])
  print(('*'*124+'\n')*10)
Course_URL = "" #@param {type:"string"}
Course_Name = Course_URL.split('/')[-1]
download_folder = "/content/drive/My Drive/Coursera-dl/"
Email = "" #@param {type:"string"}
Password = "" #@param {type:"string"}
auth = '-u %s -p %s' % (Email, Password)
CAUTH = "" #@param {type:"string"}
Specialization = False #@param {type:"boolean"}
Quality = '720p' #@param ["360p", "540p", "720p"]
try : os.mkdir(download_folder)
except FileExistsError : print("Folder exists:", download_folder)
if Specialization:
  sp = '--specialization'
  folder_path = download_folder + Course_Name
  try : os.mkdir(folder_path)
  except FileExistsError : print("Folder exists:", folder_path)
else:
  sp = ''
  folder_path = download_folder
!coursera-dl $auth --path '$folder_path' -sl en $sp --video-resolution '$Quality' -ca '$CAUTH' --download-quizzes --download-delay 1 '$Course_Name'

In [ ]:
#@markdown <br><br><center><h2>Calculate Folder Size in Google Drive</h2></center><br>
#@markdown <center><img src='https://upload.wikimedia.org/wikipedia/commons/thumb/d/da/Google_Drive_logo.png/600px-Google_Drive_logo.png' height="50" alt="Gdrive-logo"/>
#@markdown <center><h2>Paste The directory path</h2></center>
#@markdown <h5>ie. "/content/drive/My Drive/Coursera-dl"<h5><br>

import os
import matplotlib.pyplot as plt


def get_size_format(b, factor=1024, suffix="B"):
    """
    Scale bytes to its proper byte format
    e.g:
        1253656 => '1.20MB'
        1253656678 => '1.17GB'
    """
    for unit in ["", "K", "M", "G", "T", "P", "E", "Z"]:
        if b < factor:
            return f"{b:.2f}{unit}{suffix}"
        b /= factor
    return f"{b:.2f}Y{suffix}"


def get_directory_size(directory):
    """Returns the `directory` size in bytes."""
    total = 0
    try:
        # print("[+] Getting the size of", directory)
        for entry in os.scandir(directory):
            if entry.is_file():
                # if it's a file, use stat() function
                total += entry.stat().st_size
            elif entry.is_dir():
                # if it's a directory, recursively call this function
                total += get_directory_size(entry.path)
    except NotADirectoryError:
        # if `directory` isn't a directory, get the file size then
        return os.path.getsize(directory)
    except PermissionError:
        # if for whatever reason we can't open the folder, return 0
        return 0
    return total


def plot_pie(sizes, names):
    """Plots a pie where `sizes` is the wedge sizes and `names` """
    plt.pie(sizes, labels=names, autopct=lambda pct: f"{pct:.2f}%")
    plt.title("Different Sub-directory sizes in bytes")
    plt.show()


if __name__ == "__main__":
    import sys
    folder_path = "/content/drive/My Drive/Coursera-dl/matlab-programming-engineers-scientists" #@param {type:"string"}

    directory_sizes = []
    names = []
    # iterate over all the directories inside this path
    for directory in os.listdir(folder_path):
        directory = os.path.join(folder_path, directory)
        # get the size of this directory (folder)
        directory_size = get_directory_size(directory)
        if directory_size == 0:
            continue
        directory_sizes.append(directory_size)
        names.append(os.path.basename(directory) + ": " + get_size_format(directory_size))

    print("[+] Total directory size:", get_size_format(sum(directory_sizes)))
    plot_pie(directory_sizes, names)